In [ ]:
import collections
import os
import json
from difflib import SequenceMatcher as SM
from itertools import combinations
from datetime import date
import boto3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time
import pandas as pd
from collections import OrderedDict
from gspread_formatting import *
import string

In [ ]:
pubs_when_grant_primary = collections.defaultdict(dict)
pubs_when_grant_secondary_tracker = collections.defaultdict(dict)
pubs_when_grant_secondary = collections.defaultdict(dict)

fname_grant_jsons = os.listdir("/Users/luisa/pub_tracker_backup/collate_revisited_test/data_grant")
for fname_json in fname_grant_jsons:
    with open("/Users/luisa/pub_tracker_backup/collate_revisited_test/data_grant/" + fname_json, "r") as fp:
        dic_pergrant = json.load(fp)
        pubs = dic_pergrant.keys()
        grant_name = fname_json[:-5]
        pubs_when_grant_primary[grant_name] = collections.defaultdict(dict)
        pubs_when_grant_secondary[grant_name] = collections.defaultdict(dict)
        for pub in pubs:
            authors = "; ".join(dic_pergrant[pub]["authors"])
            title = dic_pergrant[pub]["title"]
            link = dic_pergrant[pub]["link"]
            date = dic_pergrant[pub]["date"]
            other_ids = ";".join(dic_pergrant[pub]["other_ids"])
            primary_awards = ";".join(dic_pergrant[pub]["primary_awards"])
            secondary_awards = ";".join(dic_pergrant[pub]["secondary_awards"])
            contact_4DN_author = dic_pergrant[pub]["4DN_member_author"]
            added_date = dic_pergrant[pub]["added"]

            pubs_when_grant_primary[grant_name][pub]['title'] = title
            pubs_when_grant_primary[grant_name][pub]['authors'] = authors
            pubs_when_grant_primary[grant_name][pub]['date'] = date
            pubs_when_grant_primary[grant_name][pub]['link'] = link
            pubs_when_grant_primary[grant_name][pub]['other_ids'] = other_ids
            pubs_when_grant_primary[grant_name][pub]['primary_awards'] = primary_awards
            pubs_when_grant_primary[grant_name][pub]['secondary_awards'] = secondary_awards
            pubs_when_grant_primary[grant_name][pub]['date_added'] = added_date
            pubs_when_grant_primary[grant_name][pub]['contact_4DN_author'] = contact_4DN_author
            
# Keeps track of the grants of the publications secondary awards 
for grant in pubs_when_grant_primary:
    for pub in pubs_when_grant_primary[grant]:
        if pubs_when_grant_primary[grant][pub]['secondary_awards'] != '':
            if ';' not in pubs_when_grant_primary[grant][pub]['secondary_awards']:
                sec_award = pubs_when_grant_primary[grant][pub]['secondary_awards']
                pubs_when_grant_secondary[sec_award][pub] = pubs_when_grant_primary[grant][pub].copy()
            else:
                sec_award_list = pubs_when_grant_primary[grant][pub]['secondary_awards'].split(';')
                for sec_award in sec_award_list:
                    pubs_when_grant_secondary[sec_award][pub] = pubs_when_grant_primary[grant][pub].copy()
                    print(pubs_when_grant_secondary[sec_award][pub])
                    print()
        pubs_when_grant_primary[grant][pub]['primary_awards'] = ' -- '


In [ ]:
print(pubs_when_grant_primary.keys())

In [ ]:
print(len(pubs_when_grant_secondary.keys()))

In [ ]:
test_grant_primary = pubs_when_grant_primary['EB021223'] # Test a specific grantpubs_when_grant_secondary

In [ ]:
test_grant_secondary = pubs_when_grant_secondary['EB021223'] 

In [ ]:
print(test_grant_secondary)

In [ ]:
print(test_grant_primary)

In [ ]:
print(len(test_grant_secondary))

In [ ]:
primary_ids = []
for pub in pubs_when_grant_primary['DK107980']:
    primary_ids.append(id(pubs_when_grant_primary['DK107980'][pub]))
    

In [ ]:
sec_ids = []
for pub in pubs_when_grant_secondary['DK107980']:
    sec_ids.append(id(pubs_when_grant_secondary['DK107980'][pub]))
    
    

In [ ]:
pri_ids_set = set(primary_ids)

In [ ]:
sec_ids_set = set(sec_ids)

In [ ]:
pri_ids_set.intersection(sec_ids_set)

In [ ]:
#print(pubs_when_grant_secondary)

In [ ]:
# Exploring the secondary awards
# for ele in test_grant_secondary.values():
#     print(ele['title'], ele['primary_awards'])

In [ ]:
# Accesses to the google sheets
s3 = boto3.resource('s3')
obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
cont = obj.get()['Body'].read().decode()
key_dict = json.loads(cont)
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
gc = gspread.authorize(creds)
book_id = '' # This is the test spreadsheet
test_spreadsheet = gc.open_by_key(book_id)

In [ ]:
input_worksheet = test_spreadsheet.worksheet("empty")

In [ ]:
input_worksheet_values = input_worksheet.get_all_values()
import pandas as pd
df = pd.DataFrame(input_worksheet_values[1:], columns=input_worksheet_values[0])
col_names = df.columns.values
from collections import OrderedDict
google_sheet = df.to_dict(orient='records', into=OrderedDict)

In [ ]:
out_worksheet = test_spreadsheet.worksheet('empty')

In [ ]:
# Comparing primary awards with existing google sheets and adding new awards
new_sheet_existing_primary = []
new_sheet_new_primary = []
for a_row in test_grant_primary.values():
    #check excel
    existing_item = [i for i in google_sheet if a_row['title'] == i['Title']]
    if existing_item:
        new_sheet_existing_primary.append(existing_item)
    else:
        new_sheet_new_primary.append(a_row)
print('{} new primary publications'.format(len(new_sheet_new_primary)))

# Comparing secondary awards with existing google sheets and adding new awards
new_sheet_existing_secondary = []
new_sheet_new_secondary = []
for a_row in test_grant_secondary.values():
    #check excel
    existing_item = [i for i in google_sheet if a_row['title'] == i['Title']]
    if existing_item:
        new_sheet_existing_secondary.append(existing_item)
    else:
        new_sheet_new_secondary.append(a_row)
print('{} new secondary publications'.format(len(new_sheet_new_secondary)))

# Adding the existing publications in the google sheets that are not in primary awards (added manually)
gs_only_items = []
for a_row in google_sheet:
    if a_row['Title'] == '' or a_row['Title'] == 'Publications in which this award is listed but it is not primary':
        continue
    existing_item_1 = [i for i in test_grant_primary.values() if a_row['Title'] == i['title']]
    existing_item_2 = [i for i in test_grant_secondary.values() if a_row['Title'] == i['title']]
    if existing_item_1 or existing_item_2:
        continue
    else:
        gs_only_items.append(a_row)

#Writting info to the google sheets
col_order = {'title': 1, 'authors': 2, 'date':3, 'contact_4DN_author':4,'primary_awards':len(col_names) - 4, 'secondary_awards':len(col_names) - 3, 'other_ids':len(col_names) - 2, 'link':len(col_names) - 1, 'date_added': len(col_names)}
gs_write = []
row = 1
fmt_bold = cellFormat(textFormat=textFormat(bold=True))
# Writting the existing info 
for r,line in enumerate(new_sheet_existing_primary):
    row = r + 1
    if row == 1: # Writes the labels of the columns (the keys)
        for c, key in enumerate(line[0]):
            col = c + 1
            gs_write.append(gspread.models.Cell(row, col, key))
    row = r + 2
    for c, key in enumerate(line[0]):
        col = c + 1
        gs_write.append(gspread.models.Cell(row, col, line[0][key]))     
# Writing existing info in gs only
for line in gs_only_items:
    row = row + 1
    for c, key in enumerate(line):
        col = c + 1
        gs_write.append(gspread.models.Cell(row, col, line[key]))

# Writting new info
for line in new_sheet_new_primary:
    row = row + 1
    for key in line:
        col = col_order[key]
        gs_write.append(gspread.models.Cell(row, col, line[key]))

#Writting message
row = row + 1
spaces = ['' for x in range(len(col_names))]
message = ['Publications in which this award is listed but it is not primary' if x == 0 else '' for x in range(len(col_names))] 

for c, txt in enumerate(spaces):
    col = c + 1
    gs_write.append(gspread.models.Cell(row,col,txt))


row = row + 1   
for c, txt in enumerate(message):
    col = c + 1
    gs_write.append(gspread.models.Cell(row,col,txt))
    format_range_message = [('A{}'.format(row),fmt_bold)]

# Writting secondary awards existing info 
for r, line in enumerate(new_sheet_existing_secondary):
    row = row + 1
    for c, key in enumerate(line[0]):
        col = c + 1
        gs_write.append(gspread.models.Cell(row, col, line[0][key]))

# Writting secondary awards new info 
for r, line in enumerate(new_sheet_new_secondary):
    row = row + 1
    for c, key in enumerate(line):
#         col = c + 1
        col = col_order[key]
        gs_write.append(gspread.models.Cell(row, col, line[key]))


# out_worksheet.update_cells(gs_write,value_input_option='USER_ENTERED')
# #Adding formatting to worksheet
# num_to_letter_dict = {x:y for x, y in zip(range(1, 27), string.ascii_uppercase)}
# num_rows = len(gs_write)//len(df.columns)
# fmt_plain = cellFormat(textFormat=textFormat(bold=False))

# range_worksheet_formatting = [('A1:{}{}'.format(num_to_letter_dict[len(df.columns)],num_rows),fmt_plain)]
# range_title_formatting = [('A1:{}1'.format(num_to_letter_dict[len(df.columns)]), fmt_bold)]

# format_cell_ranges(out_worksheet,range_worksheet_formatting) #Clean the formatting of worksheet
# format_cell_ranges(out_worksheet,format_range_message) # Make the title bold
# format_cell_ranges(out_worksheet,range_title_formatting) # Make the subtitle bold

In [ ]:
print(test_grant_primary['29045860'])

In [ ]:
for a_row in test_grant_primary.values():
    print(a_row['title'])
    break

In [ ]:
print(len(new_sheet_existing_primary))

In [ ]:
print(len(gs_only_items))

In [ ]:
print(col_order)